In [ ]:
import pandas as pd
import numpy as np
import os
import koreanize_matplotlib
import seaborn as sns
%config InlineBackend.figure_format = 'retina'  #선명하게 만들기
#한글폰트 가져오기
from matplotlib import rc
rc('font', family='NanumGothic')

# 결측치 확인하는 라이브러리
import missingno as msno
import chardet

from sklearn.preprocessing import MinMaxScaler,OneHotEncoder, StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.metrics import euclidean_distances

path = 'C:/Users/조현서/Desktop/비어플신입세션/프로젝트_1'
os.chdir(path)
print(os.getcwd())

C:\Users\조현서\Desktop\비어플신입세션\프로젝트_1


### 스케일링

코사인 유사도를 구하기 위한 데이터 정규화

범주형 특성은 다음과 같음
- 성별코드: 남성(1), 여성(2)
- 요단백: 정상(1.0) ~ (6.0) ;높을수록 나쁨
- 흡연상태: 비흡연(1), 끊음(2), 흡연중(3)
- 음주여부: 비음주(0), 음주(1)
- 청력: 정상(1), 질환의심(2)
- BMI_WC_risk: 위험도낮음(0)~(5) ;높을수록 나쁨
- WHtR_risk: 정상(1), 저체중(0)~비만(3)

그러나, 성별코드 변수를 제외하면 범주형 변수들도 미리 모두 수치화 해놓았기 떄문에 원-핫 인코딩 작업 필요하지 않음

In [17]:
df = pd.read_csv('final_ex.csv')
df_origin = df.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104819 entries, 0 to 104818
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   성별코드         104819 non-null  int64  
 1   수축기혈압        104819 non-null  float64
 2   이완기혈압        104819 non-null  float64
 3   식전혈당(공복혈당)   104819 non-null  float64
 4   혈색소          104819 non-null  float64
 5   요단백          104819 non-null  float64
 6   혈청크레아티닌      104819 non-null  float64
 7   혈청지오티(AST)   104819 non-null  float64
 8   혈청지피티(ALT)   104819 non-null  float64
 9   감마지티피        104819 non-null  float64
 10  흡연상태         104819 non-null  float64
 11  음주여부         104819 non-null  float64
 12  시력           104819 non-null  float64
 13  청력           104819 non-null  float64
 14  BMI_WC_risk  104819 non-null  int64  
 15  WHtR_risk    104819 non-null  int64  
dtypes: float64(13), int64(3)
memory usage: 12.8 MB


In [18]:
#######################################가경이행 철수행 생성
gg = {
    '성별코드': 2,
    '수축기혈압': 105,
    '이완기혈압': 70.0,
    '식전혈당(공복혈당)': 90.0,
    '혈색소': 13.75,
    '요단백': 1.0,
    '혈청크레아티닌': 1.25,
    '혈청지오티(AST)': 20.0,
    '혈청지피티(ALT)': 17.5,
    '감마지티피': 21.5,
    '흡연상태': 1,  
    '음주여부': 0,  
    '시력': 0.6,
    '청력': 1.0,
    'BMI_WC_risk': 0,
    'WHtR_risk': 1
}
cs = {
    '성별코드': 1,
    '수축기혈압': 105,
    '이완기혈압': 70.0,
    '식전혈당(공복혈당)': 90.0,
    '혈색소': 14.75,
    '요단백': 1.0,
    '혈청크레아티닌': 1.25,
    '혈청지오티(AST)': 20.0,
    '혈청지피티(ALT)': 17.5,
    '감마지티피': 37,
    '흡연상태': 1,  
    '음주여부': 0,  
    '시력': 0.6,
    '청력': 1.0,
    'BMI_WC_risk': 0,
    'WHtR_risk': 1
}
healthy= pd.DataFrame([gg, cs])

df = pd.concat([df, healthy], ignore_index=True)
df.tail()

,성별코드,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
104816,2,100.0,61.0,93.0,13.60,1.0,0.80,20.0,14.0,12.0,1.0,1.0,1.1,1.0,1,0
104817,2,98.0,67.0,96.0,13.20,3.0,1.00,21.0,14.0,25.0,1.0,0.0,1.2,1.0,1,0
104818,1,125.0,80.0,92.0,16.90,1.0,1.10,30.0,20.0,16.0,1.0,1.0,0.8,1.0,2,0
104819,2,105.0,70.0,90.0,13.75,1.0,1.25,20.0,17.5,21.5,1.0,0.0,0.6,1.0,0,1
104820,1,105.0,70.0,90.0,14.75,1.0,1.25,20.0,17.5,37.0,1.0,0.0,0.6,1.0,0,1


성별 기준으로 데이터셋 분리

In [19]:
male_group = df[df['성별코드'] == 1]
female_group = df[df['성별코드'] == 2]

In [20]:
# 나눴으므로 성별코드 제거
male_group = male_group.drop(columns=['성별코드'])
female_group = female_group.drop(columns=['성별코드'])

In [33]:
# 범주형 및 연속형 변수 정의
# categorical_columns = ['요단백', '흡연상태', '음주여부', '청력', 'BMI_WC_risk', 'WHtR_risk']
# continuous_columns = ['수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '혈색소', '혈청크레아티닌', '혈청지오티(AST)','시력', '혈청지피티(ALT)', '감마지티피']

# 1. 범주형 변수 원-핫 인코딩
#encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first') # 다중 공선성 문제를 피하기 위해 첫 번쨰 열 제거
#encoded_male_group = encoder.fit_transform(male_group[categorical_columns])
#encoded_female_group = encoder.transform(female_group[categorical_columns])

# 2. 연속형 변수 표준화
#scaler = StandardScaler()
#scaled_male_continuous = scaler.fit_transform(male_group[continuous_columns])
#scaled_female_continuous = scaler.transform(female_group[continuous_columns])

# 범주형 변수 원-핫 인코딩된 결과를 데이터프레임으로 변환
#encoded_male_df = pd.DataFrame(encoded_male_group, columns=encoder.get_feature_names_out(categorical_columns))
#encoded_female_df = pd.DataFrame(encoded_female_group, columns=encoder.get_feature_names_out(categorical_columns))

# 연속형 변수 표준화된 결과를 데이터프레임으로 변환
#scaled_male_df = pd.DataFrame(scaled_male_continuous, columns=continuous_columns)
#scaled_female_df = pd.DataFrame(scaled_female_continuous, columns=continuous_columns)

# 범주형, 연속형 변수를 결합하여 최종 데이터셋 생성
#enc_male = pd.concat([encoded_male_df, scaled_male_df], axis=1)
#enc_female = pd.concat([encoded_female_df, scaled_female_df], axis=1)

C:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [21]:
# StandardScaler로 정규화
scaler = StandardScaler()
scaled_male_data = scaler.fit_transform(male_group)
scaled_female_data = scaler.fit_transform(female_group)

# 데이터프레임으로 변환
enc_male = pd.DataFrame(scaled_male_data, columns=male_group.columns)
enc_female = pd.DataFrame(scaled_female_data, columns=female_group.columns)

일반 사용자와 건강한 행 분리

In [22]:
# 건강한 행
enc_healthy_male = enc_male[-1:]
enc_healthy_female = enc_female[-1:]
# 일반 사용자 행
enc_male = enc_male[:51095]
enc_female = enc_female[:53724]

In [23]:
# 변수별 유클리드 거리를 저장할 빈 데이터프레임 생성
uc_similarity_male = pd.DataFrame(columns=enc_male.columns)

# 각 변수별 유클리드 거리 계산
for column in enc_male.columns:
    reference_value = np.array(enc_healthy_male[column]).reshape(1, -1)
    euclidean_scores = euclidean_distances(enc_male[[column]], reference_value).flatten()
    uc_similarity_male[column] = euclidean_scores

In [24]:
uc_similarity_male

,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
0,1.258414,0.106113,0.138930,0.158573,0.0,1.522616,0.475003,0.990115,0.379034,0.000000,2.720519,0.680267,0.0,2.820355,1.317652
1,0.419471,0.000000,0.416790,0.158573,0.0,0.138420,0.345457,0.113620,0.234640,2.181657,2.720519,2.040800,0.0,1.410177,0.000000
2,2.516828,1.697804,0.625186,0.898581,0.0,0.415259,0.734096,0.892727,0.397083,2.181657,2.720519,0.544213,0.0,2.115266,0.000000
3,2.516828,0.955014,0.555720,1.532874,0.0,0.692098,0.000000,0.243471,0.072197,0.000000,0.000000,0.952373,0.0,2.115266,0.000000
4,0.755048,0.848902,0.486255,0.475720,0.0,0.968937,0.086364,0.016231,0.360985,2.181657,2.720519,0.408160,0.0,0.000000,1.317652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51090,1.174520,0.530564,0.694651,1.215728,0.0,0.692098,0.431821,0.113620,0.198542,0.000000,0.000000,2.448960,0.0,2.115266,0.000000
51091,0.587260,0.106113,0.347325,0.052858,0.0,1.522616,0.129546,0.633024,0.270739,1.090829,2.720519,1.088427,0.0,1.410177,0.000000
51092,0.167789,0.636676,0.069465,1.532874,0.0,0.415259,0.172728,0.016231,0.415133,0.000000,2.720519,2.448960,0.0,0.705089,1.317652
51093,1.342308,0.212225,0.486255,1.321443,0.0,1.245777,1.209099,2.808032,1.714678,0.000000,2.720519,1.632640,0.0,1.410177,0.000000


In [25]:
uc_similarity_male.describe()

,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
count,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000,51095.000000
mean,1.445273,0.833527,0.570629,1.009421,0.258535,0.861448,0.384141,0.520579,0.371137,0.873965,2.282425,1.284402,0.031918,1.669899,0.579643
std,0.908552,0.682101,0.846954,0.748529,1.000019,0.978842,0.967738,0.957340,0.931313,1.000012,0.999969,0.851707,1.000020,0.999992,0.825235
min,0.000000,0.000000,0.000000,0.052858,0.000000,0.138420,0.000000,0.016231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.755048,0.318338,0.208395,0.370004,0.000000,0.692098,0.086364,0.081157,0.180492,0.000000,2.720519,0.816320,0.000000,0.705089,0.000000
50%,1.258414,0.742789,0.416790,0.898581,0.000000,0.968937,0.172728,0.211008,0.306837,0.000000,2.720519,1.088427,0.000000,1.410177,0.000000
75%,2.097357,1.061127,0.694651,1.427159,0.000000,1.245777,0.345457,0.535636,0.397083,2.181657,2.720519,1.632640,0.000000,2.115266,1.317652
max,9.647840,7.427891,36.747015,10.941550,11.830417,193.095372,46.463957,42.769731,179.806567,2.181657,2.720519,25.305916,31.362659,3.525443,2.635303


In [26]:
# 변수별 유클리드 거리를 저장할 빈 데이터프레임 생성
uc_similarity_female = pd.DataFrame(columns=enc_female.columns)

# 각 변수별 유클리드 거리 계산
for column in enc_female.columns:
    reference_value = np.array(enc_healthy_female[column]).reshape(1, -1)
    euclidean_scores = euclidean_distances(enc_female[[column]], reference_value).flatten()
    uc_similarity_female[column] = euclidean_scores

In [27]:
uc_similarity_female

,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
0,1.147266,1.102393,0.224958,1.567079,0.000000,3.800461,0.260650,0.192566,0.619813,0.000000,2.731847,1.429693,0.0,0.000000,1.345029
1,0.353005,0.330718,0.000000,0.050551,0.000000,3.800461,0.521301,0.281442,0.619813,0.000000,2.731847,1.715631,0.0,0.786243,0.000000
2,4.147807,2.315025,1.799664,0.050551,0.000000,3.215775,0.065163,0.222191,0.163109,0.000000,2.731847,1.143754,0.0,2.358728,0.000000
3,0.794261,0.330718,0.974818,4.903441,0.000000,3.800461,0.130325,0.162940,0.946031,0.000000,2.731847,0.571877,0.0,0.786243,1.345029
4,1.676773,0.881914,0.074986,1.162672,0.000000,3.215775,0.195488,0.162940,0.358839,0.000000,0.000000,1.000785,0.0,0.786243,1.345029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53719,1.323768,0.000000,0.899832,0.252755,0.000000,3.215775,0.065163,0.222191,0.358839,0.000000,2.731847,2.573447,0.0,0.000000,1.345029
53720,2.029778,0.771675,0.224958,0.556060,0.000000,1.461716,0.130325,0.251817,0.293596,0.000000,2.731847,0.142969,0.0,0.786243,0.000000
53721,1.853275,0.992153,0.374930,2.578098,0.000000,3.215775,0.260650,0.014813,0.032622,3.517855,2.731847,1.429693,0.0,2.358728,0.000000
53722,0.441256,0.992153,0.224958,0.151653,0.000000,2.631088,0.000000,0.103689,0.619813,0.000000,2.731847,1.429693,0.0,0.786243,1.345029


In [28]:
uc_similarity_female.describe()

,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
count,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000,53724.000000
mean,0.916948,0.796253,0.546369,0.869274,0.314508,3.295614,0.341059,0.246998,0.624419,0.366132,2.296421,1.191312,0.023240,1.131347,0.801574
std,0.741093,0.611962,0.837566,0.759939,1.000018,0.950149,0.940062,0.970177,0.833260,1.000017,0.999970,0.777290,1.000019,1.000007,0.756001
min,0.000000,0.000000,0.000000,0.050551,0.000000,0.292343,0.000000,0.014813,0.032622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.441256,0.330718,0.224958,0.353857,0.000000,2.631088,0.130325,0.103689,0.358839,0.000000,2.731847,0.714846,0.000000,0.786243,0.000000
50%,0.706010,0.661436,0.374930,0.657162,0.000000,3.215775,0.260650,0.192566,0.554570,0.000000,2.731847,1.143754,0.000000,0.786243,1.345029
75%,1.323768,1.102393,0.674874,1.162672,0.000000,3.800461,0.390976,0.251817,0.750300,0.000000,2.731847,1.429693,0.000000,1.572485,1.345029
max,8.383865,6.504117,38.317850,10.362943,9.537484,99.689017,92.009591,186.033389,49.878645,3.517855,2.731847,26.592281,43.053301,3.931213,2.690057


In [32]:
uc_similarity_df=pd.concat([uc_similarity_male,uc_similarity_female],axis=0, ignore_index=True)

In [33]:
uc_similarity_df.describe()

,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
count,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000,104819.000000
mean,1.174485,0.814422,0.558195,0.937590,0.287223,2.109057,0.362060,0.380358,0.500954,0.613680,2.289599,1.236690,0.027470,1.393869,0.693392
std,0.868107,0.647367,0.842239,0.757641,1.000405,1.552458,0.953892,0.973587,0.891451,1.031724,0.999989,0.815740,1.000024,1.035594,0.798250
min,0.000000,0.000000,0.000000,0.050551,0.000000,0.138420,0.000000,0.014813,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.441256,0.318338,0.208395,0.353857,0.000000,0.968937,0.086364,0.103689,0.234640,0.000000,2.720519,0.714846,0.000000,0.705089,0.000000
50%,1.059015,0.742789,0.416790,0.758264,0.000000,1.799455,0.195488,0.192566,0.397083,0.000000,2.720519,1.143754,0.000000,0.786243,0.000000
75%,1.677885,1.102393,0.694651,1.321443,0.000000,3.215775,0.388639,0.311068,0.619813,1.090829,2.731847,1.632640,0.000000,2.115266,1.345029
max,9.647840,7.427891,38.317850,10.941550,11.830417,193.095372,92.009591,186.033389,179.806567,3.517855,2.731847,26.592281,43.053301,3.931213,2.690057


In [34]:
# 전처리 완료된 유클리드 거리 저장
uc_similarity_df.to_csv('uc_similarity.csv', index=False)

print("CSV 파일로 저장되었습니다.")

CSV 파일로 저장되었습니다.
